# Main UI

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import threading
import os
import subprocess
from IPython import get_ipython



CLOUD_GPU = "unknown"
SUDO = None

def getHOME_DIR():
    global CLOUD_GPU, SUDO
    home = os.path.expanduser("~/") 
    if home == "/teamspace/studios/this_studio/":
        CLOUD_GPU = "LightningAI"
        SUDO = "sudo"
        home = "/teamspace/studios/this_studio"
    elif home == "/root/":
        CLOUD_GPU = "RunPod"
        SUDO = ""
        home = "/workspace/runpod-slim"
    else:
        CLOUD_GPU = "unknown"

    return home

HOME_DIR = getHOME_DIR()
WORK_STEPS = f"{HOME_DIR}/ComfyNotebooks"

# === 出力用ウィジェット（1つだけ）===
output = widgets.Output()
display(output)

# === Notebookを実行する関数 ===
def run_notebook(filename):
    with output:
        output.clear_output()
        full_path = os.path.join(WORK_STEPS, filename)
        print(f"▶ Notebookを実行中: {full_path}\n")
        try:
            get_ipython().run_line_magic('run', full_path)
        except Exception as e:
            import traceback
            traceback.print_exc()

# === 非同期実行（UIが固まらないように）===
def run_notebook_async(filename):
    def target():
        run_notebook(filename)
    threading.Thread(target=target, daemon=True).start()

# === Notebookファイルのリストアップ ===
notebook_options = sorted([
    f for f in os.listdir(WORK_STEPS)
    if f.startswith("step") and f.endswith(".ipynb")
])

# === UI部品の作成 ===
dropdown = widgets.Dropdown(
    options=notebook_options,
    description='実行するNotebook:',
    layout=widgets.Layout(width='70%'),
    style={'description_width': 'initial'}
)

run_button = widgets.Button(
    description="実行",
    button_style='success',
    icon='play'
)

exit_button = widgets.Button(
    description="終了",
    button_style='danger',
    icon='stop'
)

# === 実行ボタンの処理 ===
def on_run_button_clicked(_):
    run_notebook(dropdown.value)

run_button.on_click(on_run_button_clicked)

# === 終了ボタンの処理 ===
def on_exit_button_clicked(_):
    with output:
        print("\n📦 使用中のStorage容量:")
        subprocess.run(["du", "-sh", "."], text=True)

        print("\n📁 サイズの大きいファイル 上位10件:")
        subprocess.run(
            "find . -type f -exec du -h {} + | sort -rh | head -n 10",
            shell=True, text=True
        )

        print("\n⚠️ Notebookのカーネルを5秒後に停止します...")

    import time
    time.sleep(5)
    get_ipython().kernel.do_shutdown(restart=False)

exit_button.on_click(on_exit_button_clicked)

# === AUTO_RUN用Notebook ===
AUTO_RUN_NOTEBOOK = "step0_initialize.ipynb"

# === 自動実行処理 ===
if AUTO_RUN_NOTEBOOK in notebook_options:
    run_notebook(AUTO_RUN_NOTEBOOK)
else:
    print(f"⚠️ 自動実行のNotebookが見つかりません: {AUTO_RUN_NOTEBOOK}")

# === UIの表示 ===
display(dropdown)
display(widgets.HBox([run_button, exit_button]))
display(output)
